# Machine Learning Final Project Notebook with DenseNet 201
## *Petals to the Metal: Flower Classification on TPU*
### *By Xuanzhi Huang, Rahul Paul*

## Step 1: Some pre-setting

* ### Package preliminary

#### Import all the packages we need.

In [ ]:
# These are some basic packages
import random, re, math, os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt


# These are for data processing
import tensorflow_addons as tfa
from kaggle_datasets import KaggleDatasets


# These are for model training
from tensorflow.keras.layers import *
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.applications import DenseNet201
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold


# These are performance metrics
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


# These are for class weights. Not used
import datetime
import tqdm
import json
from collections import Counter
import gc


# These are for model visualization. Not used
from tensorflow.keras.utils import plot_model
from IPython.display import Image

* ### Detect the hardware and tell the appropriate distribution strategy

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Make the system tune the number of threads for us
AUTO = tf.data.experimental.AUTOTUNE

* ### Configuration for image size, training epoch, number of folds, batch size, and random seed

In [ ]:
IMAGE_SIZE = [512, 512]
EPOCHS = 16
SEED = 100
FOLDS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

* ### Set the data access

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
GCS_PATH_SELECT = { 
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

* ### Add more mixed precision and/or XLA (refer to Chris Deotte's notebook)
[Rotation Augmentation GPU/TPU - [0.96+]](https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96)

In [ ]:
# Add more mixed precision and/or XLA to allow the TPU memory to handle larger batch sizes 
# and can speed up the training process
MIXED_PRECISION = False
XLA_ACCELERATE = False

if MIXED_PRECISION:
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

* ### Show all the classes we have

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose'] 

## Step 2: Set some visualization functions

* ### Set training and validation curve function to show the changes in loss and accuracy (not used)

In [ ]:
def plot_train_valid_curves(training, validation, title, subplot):
    
    if subplot % 10 == 1:
        plt.subplots(figsize = (15,15), facecolor = '#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['training', 'validation.'])

* ### Set a function to plot confusion matrix

In [ ]:
def display_confusion_matrix(cmat, score, precision, recall):
    
    plt.figure(figsize = (15, 15))
    ax = plt.gca()
    ax.matshow(cmat, cmap = 'Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict = {'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha = "left", rotation_mode = "anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict = {'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation = 45, ha = "right", rotation_mode = "anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict = {'fontsize': 18, 'horizontalalignment': 'right', 'verticalalignment': 'top', 'color': '#804040'})
    plt.show()

* ### Construct functions to show the beautiful flowers (refer to Dimitre Oliveira) (not used)
[Flower with TPUs - Advanced augmentations](https://www.kaggle.com/dimitreoliveira/flower-with-tpus-advanced-augmentations)

In [ ]:
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object:
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (for test data)
    return numpy_images, numpy_labels


def title_from_label_and_target_(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct


def display_one_flower(image, title, subplot, red = False, titlesize = 16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize = int(titlesize) if not red else int(titlesize / 1.2), color = 'red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2] + 1)


def display_batch_of_images(databatch, predictions = None):
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
    rows = int(math.sqrt(len(images)))
    cols = len(images) // rows
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot = (rows,cols,1)
    if rows < cols:
        plt.figure(figsize = (FIGSIZE, FIGSIZE / cols*rows))
    else:
        plt.figure(figsize = (FIGSIZE / rows * cols,FIGSIZE))
    
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target_(predictions[i], label)
        dynamic_titlesize = FIGSIZE * SPACING / max(rows,cols) * 40 + 3
        subplot = display_one_flower(image, title, subplot, not correct, titlesize = dynamic_titlesize)
    
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace = 0, hspace = 0)
    else:
        plt.subplots_adjust(wspace = SPACING, hspace = SPACING)
    plt.show()
    

# Visualize predictions. Images with labels telling whether prediction is true will be shown.
def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis = -1)
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', should be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower_eval(image, title, subplot, red = False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize = 14, color = 'red' if red else 'black')
    return subplot + 1

def display_9_images_with_predictions(images, predictions, labels):
    subplot = 331
    plt.figure(figsize = (13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower_eval(image, title, subplot, not correct)
        if i >= 8:
            break;
    plt.tight_layout()
    plt.subplots_adjust(wspace = 0.1, hspace = 0.1)
    plt.show()

## Step 3: Set functions to gain training set, validation set, and test set

* ### Decode images and standardize them

In [ ]:
def decode_image(image_data):
    
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.cast(image, tf.float32) / 255.0  
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    
    return image

* ### Set a function to read labeled tfrec files (i.e. training & validation set)

In [ ]:
def read_labeled_tfrecord(example):
    
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    
    return image, label



# This is for data visualization. Not used
def read_labeled_id_tfrecord(example):
    
    LABELED_ID_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, LABELED_ID_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    idnum =  example['id']
    
    return image, label, idnum

* ### Set a function to read unlabeled tfrec files (i.e. test set)

In [ ]:
def read_unlabeled_tfrecord(example):
    
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    
    return image, idnum

* ### Load image data

In [ ]:
# For best performance, read from multiple tfrec files at once
# Disregard data's order, since data will be shuffled
def load_dataset(filenames, labeled = True, ordered = False):
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False  # Disable order to increase running speed
    # Automatically interleaves reading
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # Use data in the shuffled order
    dataset = dataset.with_options(ignore_order)
    # Returns a dataset of (image, label) pairs if labeled = True (i.e. training & validation set)
    # or (image, id) pairs if labeld = False (i.e. test set)
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO)
    
    return dataset

* ### Data augmentation

#### Since the quality of images in test set are similar to images in training and validation set, to build a better model for test images, there is no need to do too much image augmentation. We think resizing, cropping, and flipping are adequate. However, for better generalization, it is reasonable to take all the augmentation listed below (including those set as comments) into account.

In [ ]:
# Randomly make some changes to the images and return the new images and labels
def data_augment(image, label):
        
    # Set seed for data augmentation
    seed = 100
    
    # Randomly resize and then crop images
    image = tf.image.resize(image, [800, 800])
    image = tf.image.random_crop(image, [512, 512, 3], seed = seed)

    # Randomly reset brightness of images
    # image = tf.image.random_brightness(image, 0.6, seed = seed)
    
    # Randomly reset saturation of images
    # image = tf.image.random_saturation(image, 3, 5, seed = seed)
        
    # Randomly reset contrast of images
    # image = tf.image.random_contrast(image, 0.3, 0.5, seed = seed)

    # Blur images
    # image = tfa.image.mean_filter2d(image, filter_shape = 10)
    
    # Randomly flip images
    image = tf.image.random_flip_left_right(image, seed = seed)
    image = tf.image.random_flip_up_down(image, seed = seed)
    
    return image, label

* ### Gain training set

In [ ]:
def get_training_dataset(dataset, do_aug = True):
    
    if do_aug: dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)  # prefetch next batch while training (autotune prefetch buffer size)
    
    return dataset

* ### Gain validation set

In [ ]:
def get_validation_dataset(dataset):
    
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    
    return dataset

* ### Gain test set

In [ ]:
def get_test_dataset(ordered = False):
    
    dataset = load_dataset(TEST_FILENAMES, labeled = False, ordered = ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    
    return dataset

* ### Count the number of images

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)


NUM_TRAINING_IMAGES = int(count_data_items(TRAINING_FILENAMES) * (FOLDS - 1.) / FOLDS)
NUM_VALIDATION_IMAGES = int(count_data_items(TRAINING_FILENAMES) * (1. / FOLDS))
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

* ### Show example augmentation

In [ ]:
row = 3
col = 4
all_elements = get_training_dataset(load_dataset(TRAINING_FILENAMES),do_aug = False).unbatch()
one_element = tf.data.Dataset.from_tensors(next(iter(all_elements)))
# Map the images to the data augmentation function for image processing
augmented_element = one_element.repeat().map(data_augment).batch(row * col)
augmented_element = augmented_element.prefetch(AUTO)

for (img, label) in augmented_element:
    plt.figure(figsize = (15, int(15 * row / col)))
    for j in range(row * col):
        plt.subplot(row, col, j + 1)
        plt.axis('off')
        plt.imshow(img[j, ])
    plt.show()
    break

## Step 4: Build the model and make prediction

* ### Customize learning rate scheduler and visualize it

In [ ]:
def lrfn(epoch):
    
    LR_START = 0.00001
    LR_MAX = 0.00005 * strategy.num_replicas_in_sync
    LR_MIN = 0.00001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_EXP_DECAY = .8
    
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY ** (epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

# Visualization changes in learning rate
rng = [i for i in range(25 if EPOCHS < 25 else EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

* ### Build the model and load it into TPU, make prediction (refer to Chris Deotte)
[Rotation Augmentation GPU/TPU - [0.96+]](https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96/notebook)

#### Set all the layers trainable and conduct cross validation for training and prediction. If set some layers untrainable, the accuracy grows too slow, so we don't do that.

In [ ]:
def get_model():
    
    with strategy.scope():

        rnet = DenseNet201(
            input_shape = (512, 512, 3),
            weights = 'imagenet',  # Use the preset parameters of ImageNet
            include_top = False
        )

        rnet.trainable = True
        #for i in range(len(rnet.layers) - 2, len(rnet.layers)):
        #    rnet.layers[i].trainable = True

        model = tf.keras.Sequential([
            rnet,
            tf.keras.layers.GlobalAveragePooling2D(),
            #tf.keras.layers.Dense(64, activation = 'relu', dtype = 'float32'),
            #tf.keras.layers.Dropout(0.5),
            #tf.keras.layers.Dense(32, activation = 'relu'),
            #tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(len(CLASSES), activation = 'softmax', dtype = 'float32')
        ])

        model.compile(
            optimizer = 'adam',
            # For multiclassification, we can use cross entropy or sparse cross entropy as our loss function 
            # These two cross entropy are the same in essence, but they are applied in different scenarios
            # If our target is one-hot encoded, it is better to use cross entropy
            # If our target is an integer, sparse cross entropy is a better choice, and this is our case
            loss = 'sparse_categorical_crossentropy', 
            metrics = ['sparse_categorical_accuracy']
        )

        model.summary()
        # Save the model
        return model
        
        
def train_cross_validate(folds = 5):
    
    histories = []
    models = []
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3)
    kfold = KFold(folds, shuffle = True, random_state = SEED)
    for f, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
        print(); print('#' * 25)
        print('### FOLD',f + 1)
        print('#' * 25)
        train_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[trn_ind]['TRAINING_FILENAMES']), labeled = True)
        val_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES']), labeled = True, ordered = True)
        model = get_model()
        history = model.fit(
            get_training_dataset(train_dataset), 
            steps_per_epoch = STEPS_PER_EPOCH,
            epochs = EPOCHS,
            callbacks = [lr_callback],
            validation_data = get_validation_dataset(val_dataset),
            verbose = 2
        )
        models.append(model)
        histories.append(history)
    
    return histories, models


def train_and_predict(folds = 5):
    
    test_ds = get_test_dataset(ordered = True)  # Since we are splitting the dataset and iterating separately on images and ids, order matters.
    test_images_ds = test_ds.map(lambda image, idnum: image)
    histories, models = train_cross_validate(folds = folds)
    # Get the mean probability of the folds models and the predicted labels
    probabilities = np.average([models[i].predict(test_images_ds) for i in range(folds)], axis = 0)
    predictions = np.argmax(probabilities, axis = -1)
    test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
    test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')
    np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt = ['%s', '%d'], delimiter = ',', header = 'id,label', comments = '')

    return histories, models
    
# run train and predict
histories, models = train_and_predict(folds = FOLDS)

* ### Check model's performance on validation set

#### Draw the confusion matrix, compute F1 score, precision, and recall

In [ ]:
all_labels = []; all_prob = []; all_pred = []
kfold = KFold(FOLDS, shuffle = True, random_state = SEED)
for j, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
    print('Inferring fold', j + 1, 'validation images...')
    VAL_FILES = list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES'])
    NUM_VALIDATION_IMAGES = count_data_items(VAL_FILES)
    cmdataset = get_validation_dataset(load_dataset(VAL_FILES, labeled = True, ordered = True))
    images_ds = cmdataset.map(lambda image, label: image)
    labels_ds = cmdataset.map(lambda image, label: label).unbatch()
    all_labels.append(next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy()) # get everything as one batch
    prob = models[j].predict(images_ds)
    all_prob.append(prob)
    all_pred.append(np.argmax(prob, axis = -1))
cm_correct_labels = np.concatenate(all_labels)
cm_probabilities = np.concatenate(all_prob)
cm_predictions = np.concatenate(all_pred)

cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
score = f1_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
precision = precision_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
recall = recall_score(cm_correct_labels, cm_predictions, labels = range(len(CLASSES)), average = 'macro')
display_confusion_matrix(cmat, score, precision, recall)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall)); print()